<a href="https://colab.research.google.com/github/JoanChege/CHATBOT1/blob/main/chatbottrial4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.DATA PREPARATION

In [ ]:
#first we need to import the required packages
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#we load the json file and extract the required data.
#with open('/content/dataset.json') as file:
   # data = json.load(file)

words=[] #word list
#empty list
classes = []
documents = []
ignore_words = ['?', '!']

data = open('/content/dataset.json').read()
intents  = json.loads(data)

Iterates over each intent in the 'intents' list.
For each intent, it iterates over the 'patterns' list associated with that intent.
It tokenizes each pattern (sentence) using the nltk library's 'word_tokenize' function and converts the sentence into a list of words.
It then extends the 'words' list with these newly tokenized words.
It creates a tuple (wordList, intent['tag']) and appends it to the 'documents' list.
It also checks if the tag of the current intent is already in the 'classes' list. If it's not, it appends the tag to the 'classes' list.

In [ ]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    #we take each word and tokenize it
    wordlist = nltk.word_tokenize(pattern)
    words.extend(wordlist)

    #add documents
    documents.append((wordlist, intent['tag']))

    #we add classes to our class list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

the first line takes each word in the words list and reduces the word to its base form by lemmatizing the word.
the second line sorts the words list and removes and duplicates. the sorting is in ascending order.
the next line sorts the classes list and removes any duplicate classes.
the pickle dump saves the words and classes to a file that is the words.pkl and classes.pkl and wb is the parameter that indicates that the file is bwing opened for writing in binary mode.
the above steps are used to convert the words into numerical form.

In [ ]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_words]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [ ]:
print(data)

In [ ]:
from nltk.tokenize import word_tokenize

#intialize the training data
training = []
output_empty = [0] * len(classes)
X_train = []
Y_train = []

for doc in documents:
  #intialize bag of words and generate for each document
  bow = []
         #words = word_tokenize(doc[0])
  #list of the tokenized words for the pattern
  pattern_words = doc[0]
  #lemmaioze each word
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
  #create the bag of words array with , if the word match found in current pattern
  for wordlist in words:
    bow.append(1) if wordlist in pattern_words else bow.append(0)

  #output is 0 for each tag, and 1 for current tag
  #this create the output vector
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1


  #add the bag of words and output row to the training data
  #num_samples = 100
  #for i in range(num_samples):
  #training.append([bow,  output_row])
  training = np.column_stack((training,[bow, output_row])) if len(training) else np.array([bow, output_row])


  #shuffle the features and turn into np.array
  random.shuffle(training)
  #training = np.array(training) #could add ,dtype=object

  # Convert the training data to a list from a numpy array
  #training = training.tolist()
X_train = list(training[:,0].reshape(-1,1))
Y_train = list(training[:,1].reshape(-1,1))
print("The training data has been created")



The training data has been created


<ipython-input-26-0dc80b302474>:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.column_stack((training,[bow, output_row])) if len(training) else np.array([bow, output_row])
/usr/local/lib/python3.10/dist-packages/numpy/lib/shape_base.py:652: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(v)


In [ ]:
print(data)

Split the data into traing and testing sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.1)

Modelling using LSTM

First layer contains 128 neurons, second layer contains 64 neurons and 3rd output layer contains number of neurons equal to number of intents to predict output intent with softmax function

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, GRU

In [ ]:
#create model of 3 layers.
#model = Sequential()
#model.add(Dense(128, input_shape=(len(X_train[0]),),activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(len(Y_train[0]), activation='softmax'))


In [ ]:
#embedding - Bi-LSTM - Dense - Dense
model = Sequential([
                 tf.keras.layers.Embedding(10000, 64),
                 tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                 tf.keras.layers.Dense(64, activation = 'relu'),
                 tf.keras.layers.Dense(7, activation = 'softmax' )
])


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          640000    
                                                                 
 bidirectional (Bidirection  (None, 128)               66048     
 al)                                                             
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 7)                 455       
                                                                 
Total params: 714759 (2.73 MB)
Trainable params: 714759 (2.73 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
